In [14]:
import pandas as pd
from datetime import timedelta
import numpy as np

In [15]:
flights_df = pd.read_csv('Datasets/File_1.csv')
flights_df = flights_df.drop('Unnamed: 0', axis=1)

C:\Users\dave\AppData\Local\Temp\ipykernel_2688\928529073.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  flights_df = pd.read_csv('Datasets/File_1.csv')


In [16]:
# Convert 'timestamp' and 'last_position' to datetime
flights_df['timestamp'] = pd.to_datetime(flights_df['timestamp'])
flights_df['last_position'] = pd.to_datetime(flights_df['last_position'])
flights_df

,timestamp,icao24,latitude,longitude,ground_speed,track,vertical_rate,callsign,onground,alert,spi,squawk,baro_altitude,altitude,last_position,hour
0,2022-10-01 17:22:01,4853d1,52.304360,4.766124,NaN,NaN,NaN,TRA13Y,True,False,False,NaN,NaN,NaN,2022-10-01 17:22:00.300,1664636400
1,2022-10-01 17:22:02,4853d1,52.304360,4.766124,NaN,NaN,NaN,TRA13Y,True,False,False,NaN,NaN,NaN,2022-10-01 17:22:00.300,1664636400
2,2022-10-01 17:22:03,4853d1,52.304360,4.766124,NaN,NaN,NaN,TRA13Y,True,False,False,NaN,NaN,NaN,2022-10-01 17:22:00.300,1664636400
3,2022-10-01 17:22:04,4853d1,52.304360,4.766124,NaN,NaN,NaN,TRA13Y,True,False,False,NaN,NaN,NaN,2022-10-01 17:22:00.300,1664636400
4,2022-10-01 17:22:05,4853d1,52.304360,4.766124,NaN,NaN,NaN,TRA13Y,True,False,False,NaN,NaN,NaN,2022-10-01 17:22:00.300,1664636400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1083769,2022-10-30 22:09:35,484133,52.305061,4.769783,141.226646,183.247343,0.0,KLM76T,False,False,False,7124.0,40000.0,NaN,2022-10-30 22:04:39.505,1667163600
1083770,2022-10-30 22:09:36,484133,52.305061,4.769783,141.226646,183.247343,0.0,KLM76T,False,False,False,7124.0,40000.0,NaN,2022-10-30 22:04:39.505,1667163600
1083771,2022-10-30 22:09:37,484133,52.305061,4.769783,141.226646,183.247343,0.0,KLM76T,False,False,False,7124.0,40000.0,NaN,2022-10-30 22:04:39.505,1667163600
1083772,2022-10-30 22:09:38,484133,52.305061,4.769783,141.226646,183.247343,0.0,KLM76T,False,False,False,7124.0,40000.0,NaN,2022-10-30 22:04:39.505,1667163600


In [17]:
flights_df.drop_duplicates(subset=['icao24', 'last_position'], inplace = True)
flights_df = flights_df[(flights_df['longitude'] <= 20) & (flights_df['longitude'] >= -20)]
flights_df = flights_df[(flights_df['baro_altitude'] >= 10000) & (flights_df['baro_altitude'] <= 50000)]
print(flights_df.shape)
flights_df = flights_df[(flights_df['vertical_rate'] >= -4500) & (flights_df['vertical_rate'] <= 4500)]
print(flights_df.shape)
flights_df = flights_df[['timestamp', 'icao24', 'latitude', 'longitude', 'ground_speed', 'track', 'vertical_rate', 'baro_altitude']]
flights_df

(648854, 16)
(648573, 16)


,timestamp,icao24,latitude,longitude,ground_speed,track,vertical_rate,baro_altitude
929,2022-10-01 17:37:30,4853d1,52.044564,4.795846,295.413015,166.293039,2560.0,10025.0
930,2022-10-01 17:37:31,4853d1,52.043260,4.796317,295.413015,166.293039,2688.0,10050.0
931,2022-10-01 17:37:32,4853d1,52.041910,4.796866,295.651567,166.104763,2752.0,10100.0
932,2022-10-01 17:37:33,4853d1,52.040467,4.797494,296.863349,165.963757,2816.0,10150.0
933,2022-10-01 17:37:34,4853d1,52.039117,4.798043,297.107468,165.776669,2752.0,10200.0
...,...,...,...,...,...,...,...,...
1082899,2022-10-30 21:55:02,484133,52.339519,4.709957,141.226646,183.247343,0.0,40000.0
1082983,2022-10-30 21:56:26,484133,52.330330,4.716034,141.226646,183.247343,0.0,40000.0
1082999,2022-10-30 21:56:42,484133,52.329014,4.718380,141.226646,183.247343,0.0,40000.0
1083022,2022-10-30 21:57:05,484133,52.328419,4.721928,141.226646,183.247343,0.0,18300.0


In [18]:
flights_df.isnull().sum(), flights_df.shape

(timestamp        0
 icao24           0
 latitude         0
 longitude        0
 ground_speed     0
 track            0
 vertical_rate    0
 baro_altitude    0
 dtype: int64,
 (648573, 8))

In [19]:
flights_df.dropna(inplace = True)
flights_df.isnull().sum(), flights_df.shape

(timestamp        0
 icao24           0
 latitude         0
 longitude        0
 ground_speed     0
 track            0
 vertical_rate    0
 baro_altitude    0
 dtype: int64,
 (648573, 8))

In [20]:
flights_df_sorted = flights_df.sort_values(by=['icao24', 'timestamp'])
flights_df_sorted

,timestamp,icao24,latitude,longitude,ground_speed,track,vertical_rate,baro_altitude
1081344,2022-10-30 21:29:07,484133,50.280304,2.691000,524.069199,32.295105,0.0,34000.0
1081345,2022-10-30 21:29:08,484133,50.282329,2.692987,524.069199,32.295105,0.0,34000.0
1081346,2022-10-30 21:29:09,484133,50.284744,2.694903,524.069199,32.295105,0.0,34000.0
1081347,2022-10-30 21:29:10,484133,50.286705,2.697069,524.069199,32.295105,0.0,34000.0
1081348,2022-10-30 21:29:11,484133,50.286705,2.697069,524.069199,32.295105,0.0,34000.0
...,...,...,...,...,...,...,...,...
704309,2022-10-24 07:05:06,503d40,43.879028,16.596126,468.208950,141.416637,64.0,39000.0
704311,2022-10-24 07:05:08,503d40,43.875778,16.599703,468.208950,141.416637,64.0,39000.0
704313,2022-10-24 07:05:10,503d40,43.871155,16.604813,468.208950,141.416637,64.0,39000.0
704316,2022-10-24 07:05:13,503d40,43.867151,16.609235,468.208950,141.416637,64.0,39000.0


In [21]:
# Downsampling the dataset by keeping one in every 60 data points
downsampled_df = flights_df_sorted.iloc[::60, :]

In [22]:
# Function to find the row with the closest timestamp to the target time
def find_first_future_row(df, target_time, icao24):
    future_rows = df[(df['icao24'] == icao24) & (df['timestamp'] >= target_time) & (df['timestamp'] <= target_time + timedelta(minutes=2))]
    return future_rows.iloc[0] if not future_rows.empty else None

# Testing the function on a small subset of the data
subset_df = downsampled_df

# Adding new columns for future position and altitude
subset_df['latitude_in_10min'] = np.nan
subset_df['longitude_in_10min'] = np.nan
subset_df['baro_altitude_in_10min'] = np.nan

# Iterating over each row in the subset
for index, row in subset_df.iterrows():
    future_time = row['timestamp'] + timedelta(minutes=10)
    future_row = find_first_future_row(subset_df, future_time, row['icao24'])
    if future_row is not None:
        subset_df.at[index, 'latitude_in_10min'] = future_row['latitude']
        subset_df.at[index, 'longitude_in_10min'] = future_row['longitude']
        subset_df.at[index, 'baro_altitude_in_10min'] = future_row['baro_altitude']

# Displaying the modified subset to check the results
subset_df.head()
subset_df.isna().sum()

C:\Users\dave\AppData\Local\Temp\ipykernel_2688\3311161469.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['latitude_in_10min'] = np.nan
C:\Users\dave\AppData\Local\Temp\ipykernel_2688\3311161469.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['longitude_in_10min'] = np.nan
C:\Users\dave\AppData\Local\Temp\ipykernel_2688\3311161469.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

timestamp                    0
icao24                       0
latitude                     0
longitude                    0
ground_speed                 0
track                        0
vertical_rate                0
baro_altitude                0
latitude_in_10min         1414
longitude_in_10min        1414
baro_altitude_in_10min    1414
dtype: int64

In [23]:
subset_df.dropna(inplace = True)

C:\Users\dave\AppData\Local\Temp\ipykernel_2688\1856307613.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df.dropna(inplace = True)


In [24]:
subset_df['Climbing'] = subset_df['vertical_rate'] > 0
subset_df['Descending'] = subset_df['vertical_rate'] < 0
subset_df['Cruise'] = subset_df['vertical_rate'] == 0

C:\Users\dave\AppData\Local\Temp\ipykernel_2688\1686109439.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['Climbing'] = subset_df['vertical_rate'] > 0
C:\Users\dave\AppData\Local\Temp\ipykernel_2688\1686109439.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['Descending'] = subset_df['vertical_rate'] < 0
C:\Users\dave\AppData\Local\Temp\ipykernel_2688\1686109439.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

In [25]:
subset_df

,timestamp,icao24,latitude,longitude,ground_speed,track,vertical_rate,baro_altitude,latitude_in_10min,longitude_in_10min,baro_altitude_in_10min,Climbing,Descending,Cruise
1081344,2022-10-30 21:29:07,484133,50.280304,2.691000,524.069199,32.295105,0.0,34000.0,51.483078,3.806911,17550.0,False,False,True
1081404,2022-10-30 21:30:07,484133,50.403076,2.812572,523.759989,32.446022,-64.0,34000.0,51.590744,3.887824,15500.0,False,True,False
1081464,2022-10-30 21:31:07,484133,50.525436,2.934795,525.140480,32.479560,-960.0,33000.0,51.692210,3.963928,13425.0,False,True,False
1081526,2022-10-30 21:32:09,484133,50.651023,3.060657,530.969409,32.462916,-896.0,32025.0,51.791087,4.038773,11300.0,False,True,False
1081586,2022-10-30 21:33:09,484133,50.777573,3.188336,535.111675,32.561525,-1984.0,30625.0,51.883118,4.107254,10500.0,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
703333,2022-10-24 06:48:46,503d40,45.536840,14.723153,466.511119,141.877264,0.0,39025.0,44.368515,16.049717,39000.0,False,False,True
703395,2022-10-24 06:49:48,503d40,45.431396,14.840437,464.490718,141.995192,0.0,39000.0,44.368515,16.049717,39000.0,False,False,True
703457,2022-10-24 06:50:50,503d40,45.326936,14.956010,461.069011,142.135915,0.0,39000.0,44.177608,16.264147,39000.0,False,False,True
703521,2022-10-24 06:51:54,503d40,45.218857,15.075008,466.068262,142.146580,-64.0,38975.0,44.177608,16.264147,39000.0,False,True,False


In [26]:
subset_df.to_csv('Datasets/Cleaned_prepared_data.csv')